In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from PIL import Image

# 加载TF模块
from tensorflow.keras import layers,Sequential,losses,optimizers,datasets

# 12. 自编码器
前面我们介绍了在给出样本及其的标签的情况下，神经网络如何学习的算法，这类算法需要学习的是在给定样本$x$下的条件概率$P(y|x)$。现在获取海量的样本数据是相对容易的，困难的是获取这些数据所对应的标签信息。例如机器翻译，除了收集源语言的对话文本外，还需要待翻译的目标语言文本数据。数据的标注工作目前主要还是依赖人的`先验知识`(Prior Knowledge)来完成。深度学习所需要的数据规模一般非常大，这种强依赖人工完成数据标注的方式代价较高，而且不可避免地引入标注人员的主观先验偏差。

面对海量的无标注数据$x$，有没有办法能够从中学习到数据的分布$P(x)$的算法？这就是本章要介绍的`无监督学习`(Unsupervised Learning)算法。特别地，如果算法把$x$作为监督信号来学习，这类算法称为`自监督学习`(Self-supervised Learning)，本章要介绍的自编码器算法就是属于自监督学习范畴。

### 12.1 自编码器原理
让我们来考虑`监督学习`中神经网络的功能：
+ $\displaystyle o = f_{\theta}, x \in R^{d_{in}}, o \in R^{d_{out}}$

$d_{in}$是输入的特征向量长度，$d_{out}$是网络输出的向量长度。对于分类问题，网络模型通过把长度为$d_{in}$输入特征向量$x$变换到长度为$d_{out}$的输出向量$o$，这个过程可以看成是`特征降维`。最常见的降维算法有`主成分分析法`(Principal components analysis，简称PCA)，通过对协方差矩阵进行特征分解而得到数据的主要成分，但是`PCA`本质上是一种线性变换，提取特征的能力极为有限。

能不能利用神经网络的强大非线性表达能力去学习到低维的数据表示呢？问题的关键在于，训练神经网络一般需要一个显式的标签数据(或监督信号)，但是无监督的数据没有额外的标注信息，只有数据$x$本身。

于是，我们尝试着利用数据$x$本身作为监督信号来指导网络的训练，即希望神经网络能够学习到映射$f_{\theta}:x\to x$。我们把网络$f_{\theta}$切分为两个部分：
+ 前面的子网络尝试学习映射关系:$g_{\theta1}:x\to z$
+ 后面的子网络尝试学习映射关系:$h_{\theta2}:z\to x$

如`图12.1`所示：

<img src="images/12_01.png" style="width:200px;"/>

我们把$g_{\theta1}$看成一个数据`编码`(Encode)的过程，把高维度的输入$x$编码成低维度的隐变量$z$(Latent Variable，或隐藏变量)，称为`Encoder`网络(编码器)；$h_{\theta2}$看成数据`解码`(Decode)的过程，把编码过后的输入$z$解码为高维度的$x$，称为`Decoder`网络(解码器)。

编码器和解码器共同完成了输入数据$x$的编码和解码过程，我们把整个网络模型$f_{\theta}$叫做`自编码器`(Auto-Encoder)。如果使用深层神经网络来参数化$g_{\theta1}$和$h_{\theta2}$函数，则称为`深度自编码器`(Deep Auto-encoder)，如`图12.2`所示。

<img src="images/12_02.png" style="width:500px;"/>

自编码器能够将输入变换到隐藏向量$z$，并通过解码器`重建`(Reconstruct，或恢复)出$x$。我们希望解码器的输出能够完美地或者近似恢复出原来的输入，即$\bar{x} \approx x$，那么，自编码器的优化目标可以写成：
+ $\text{Minimize} \mathcal{L} = \text{dist}(x,\bar{x})$
+ $\bar{x} = h_{\theta2}\big(g_{\theta1}(x)\big)$

其中$\text{dist}(x,\bar{x})$表示$x$和$\bar{x}$的距离度量，称为`重建误差函数`。最常见的度量方法有欧氏距离：
+ $\mathcal{L} = \displaystyle\sum_{i}(x_i-\bar{x}_i)^2$

它和均方误差原理上是等价的。自编码器网络和普通的神经网络并没有本质的区别，只不过训练的监督信号由标签$y$变成了自身$x$。借助于深层神经网络的非线性特征提取能力，自编码器可以获得良好的数据表示，相对于`PCA`等线性方法，自编码器性能更加优秀。

在`图12.3(a)`中，第1行是随机采样自测试集的真实MNIST手写数字图片，第2、3、4行分别是基于长度为30的隐藏向量，使用`自编码器`、`Logistic PCA`和`标准PCA`算法恢复出的重建样本图片；在`图12.3(b)`中，第1行为真实的人像图片，第2、3行分别是基于长度为30的隐藏向量，使用`自编码器`和`标准PCA`算法恢复出的重建样本。可以看到，使用深层神经网络的自编码器重建出图片还原度较高。

<img src="images/12_03.png" style="width:500px;"/>

## 12.2 Fashion MNIST图片重建实战
我们基于`Fashsion MNIST`数据集进行图片重建实战。

### 12.2.1 Fashion MNIST数据集
`Fashion MNIST`包含了10类不同类型的衣服、鞋子、包等灰度图片，图片大小为$28\times28$，共70000张图片，其中60000张用于训练集，10000张用于测试集，如`图12.4`所示。

<img src="images/12_04.png" style="width:350px;"/>

In [2]:
batchsz = 512
h_dim = 20
lr = 1e-3

# 加载Fashion MNIST图片数据集
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# 归一化
x_train, x_test = x_train.astype(np.float32)/255., x_test.astype(np.float32)/255.

# 只需要通过图片数据即可构建数据集对象，不需要标签
train_db = tf.data.Dataset.from_tensor_slices(x_train)
train_db = train_db.shuffle(batchsz * 5).batch(batchsz)

# 构建测试集对象
test_db = tf.data.Dataset.from_tensor_slices(x_test)
test_db = test_db.batch(batchsz)

### 12.2.2 编码器/解码器
我们利用编码器将输入图片$x \in R^{784}$降维到较低维度的隐藏向量：$h \in R^{20}$，并基于隐藏向量$h$利用解码器重建图片。自编码器模型如`图12.5`所示：

<img src="images/12_05.png" style="width:500px;"/>

编码器由3层全连接层网络组成，输出节点数分别为256、128、20；解码器同样由3层全连接网络组成，输出节点数分别为128、256、784。

首先是编码器子网络的实现。利用3层的神经网络将长度为784的图片向量数据依次降维到256、128，最后降维到$h_{dim}$维度，每层使用`ReLU`激活函数，最后一层不使用激活函数。

然后再来创建解码器子网络，这里基于隐藏向量$h_{dim}$依次升维到128、256、784长度，除最后一层，激活函数使用`ReLU`函数。解码器的输出为784长度的向量，通过`Reshape`操作恢复为图片矩阵。

In [3]:
class AE(keras.Model):
    # 自编码器模型类，包含了Encoder和Decoder 2个子网络
    def __init__(self):
        super(AE, self).__init__()
        # Encoders
        self.encoder = Sequential([
            layers.Dense(256, activation=tf.nn.relu),
            layers.Dense(128, activation=tf.nn.relu),
            layers.Dense(64, activation=tf.nn.relu),
            layers.Dense(h_dim)
        ])
        # Decoders
        self.decoder = Sequential([
            layers.Dense(64, activation=tf.nn.relu),
            layers.Dense(128, activation=tf.nn.relu),
            layers.Dense(256, activation=tf.nn.relu),
            layers.Dense(784)
        ])

    def call(self, inputs, training=None):
        # [b, 784] => [b, 20]
        h = self.encoder(inputs)
        # [b, 20] => [b, 784]
        x_hat = self.decoder(h)
        return x_hat

### 12.2.5 网络训练
自编码器的训练过程与分类器的基本一致，通过误差函数计算出重建向量$\bar{x}$与原始输入向量$x$之间的距离，再利用`TensorFlow`的自动求导机制同时求出`encoder`和`decoder`的梯度，循环更新即可：

In [4]:
# 创建网络对象
model = AE()
# 指定输入大小
model.build(input_shape=(4, 784))
# 打印网络信息
model.summary()
# 创建优化器，并设置学习率
optimizer = optimizers.Adam(lr=lr)

Model: "ae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  243412    
_________________________________________________________________
sequential_1 (Sequential)    multiple                  244176    
Total params: 487,588
Trainable params: 487,588
Non-trainable params: 0
_________________________________________________________________


这里固定训练100个`Epoch`，每次通过前向计算获得重建图片向量，并利用`tf.nn.sigmoid_cross_entropy_with_logits`损失函数计算重建图片与原始图片直接的误差，实际上利用`MSE`误差函数也是可行的：

In [5]:
def save_images(imgs, name):
    new_im = Image.new('L', (280, 280))
    index = 0
    for i in range(0, 280, 28):
        for j in range(0, 280, 28):
            im = imgs[index]
            im = Image.fromarray(im, mode='L')
            new_im.paste(im, (i, j))
            index += 1
    new_im.save(name)

# 训练100个Epoch
for epoch in range(100): 
    # 遍历训练集
    for step, x in enumerate(train_db): 
        # 打平，[b, 28, 28] => [b, 784]
        x = tf.reshape(x, [-1, 784])
        # 构建梯度记录器
        with tf.GradientTape() as tape:
            # 前向计算获得重建的图片
            x_rec_logits = model(x)
            # 计算重建图片与输入之间的损失函数
            rec_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=x_rec_logits)
            # 计算均值
            rec_loss = tf.reduce_mean(rec_loss)
        # 自动求导，包含了2个子网络的梯度
        grads = tape.gradient(rec_loss, model.trainable_variables)
        # 自动更新，同时更新 2 个子网络
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # 间隔性打印训练误差
    print(epoch, step, float(rec_loss))
    # 重建图片，从测试集采样一批图片
    x = next(iter(test_db))
    logits = model(tf.reshape(x, [-1, 784]))
    x_hat = tf.sigmoid(logits)
    # [b, 784] => [b, 28, 28]
    x_hat = tf.reshape(x_hat, [-1, 28, 28])
    # [b, 28, 28] => [2b, 28, 28]
    x_concat = tf.concat([x, x_hat], axis=0)
    x_concat = x_hat
    x_concat = x_concat.numpy() * 255.
    x_concat = x_concat.astype(np.uint8)
    # save_images(x_concat, '/home/alex/temp/rec_epoch_%d.png'%epoch)

0 117 0.33658719062805176
1 117 0.3122255504131317
2 117 0.30539190769195557
3 117 0.3164972960948944
4 117 0.29966506361961365
5 117 0.3028705418109894
6 117 0.29469549655914307
7 117 0.28039780259132385
8 117 0.29256802797317505
9 117 0.3064577281475067
10 117 0.28608107566833496
11 117 0.2972445785999298
12 117 0.27093005180358887
13 117 0.29275402426719666
14 117 0.2827445864677429
15 117 0.29868945479393005
16 117 0.2867629826068878
17 117 0.29393208026885986
18 117 0.285297691822052
19 117 0.2784631550312042
20 117 0.28337258100509644
21 117 0.27688172459602356
22 117 0.2894863188266754
23 117 0.282692015171051
24 117 0.2930589020252228
25 117 0.2911820709705353
26 117 0.2721794843673706
27 117 0.27247190475463867
28 117 0.2602325677871704
29 117 0.27335622906684875
30 117 0.27941298484802246
31 117 0.28101953864097595
32 117 0.28674423694610596
33 117 0.29272356629371643
34 117 0.2892412841320038
35 117 0.273915559053421
36 117 0.2757524251937866
37 117 0.2720988690853119
38 117

图片重建的效果如`图12.6`、`图12.7`、`图12.8`所示，其中每张图片的左边5列为真实图片，右边 5 列为对应的重建图片。可以看到，随着训练的进行，重建图片边缘越来越清晰。

<img src="images/12_06.png" style="width:600px;"/>

## 12.3 自编码器变种
一般而言，自编码器网络的训练较为稳定，但是由于损失函数是直接度量重建样本与真实样本的底层特征之间的距离，而不是评价重建样本的逼真度和多样性等抽象指标，因此在某些任务上效果一般，如图片重建，容易出现重建图片边缘模糊，逼真度相对真实图片仍有不小差距。为了尝试让自编码器学习到数据的真实分布，产生了一系列的自编码器变种网络。本节介绍几种典型的自编码器变种模型。

### 12.3.1 Denoising Auto-Encoder
为了防止神经网络记忆住输入数据的底层特征，`Denoising Auto-Encoders`给输入数据添加随机的噪声扰动，如给输入$x$添加采样自高斯分布的噪声$\epsilon$：
+ $\tilde{x} = x + \epsilon, \epsilon \approx \mathcal{N}(0, \text{var})$

添加噪声后，网络需要从$\tilde{x}$学习到数据的真实隐藏变量$z$，并还原出原始的输入$x$，如`图12.9`所示：

<img src="images/12_09.png" style="width:600px;"/>

模型的优化目标为：
+ $\displaystyle\theta^{*} = \underbrace{\arg\min}_{\theta}\text{dist}\bigg(h_{\theta2}\big(g_{\theta1}(\tilde{x}),x\big)\bigg)$

### 12.3.2 Dropout Auto-Encoder
自编码器网络同样面临过拟合的风险，`Dropout Auto-Encoder`通过随机断开网络的连接防止过拟合。`Dropout Auto-Encoder`实现非常简单，通过在网络层中插入`Dropout`层即可实现网络连接的随机断开。

### 12.3.3 Adversarial Auto-Encoder
为了能够方便地从某个已知的先验分布中$p(z)$采样隐藏变量$z$，方便利用$p(z)$来重建输入，`对抗自编码器`(Adversarial Auto-Encoder)利用额外的`判别器网络`(Discriminator，简称D网络)来判定降维的隐藏变量$z$是否采样自先验分布$p(z)$，如`图12.10`所示。

<img src="images/12_10.png" style="width:500px;"/>

判别器网络的输出为一个属于$[0,1]$区间的变量，表征隐藏向量是否采样自先验分布$p(z)$：
+ 所有采样自先验分布$p(z)$的$z$标注为真
+ 采样自编码器的条件概率$q(z|x)$的$z$标注为假

通过这种方式训练，除了可以重建样本，还可以约束条件概率分布$q(z|x)$逼近先验分布$p(z)$。

对抗自编码器是从下一章要介绍的生成对抗网络算法衍生而来，在学习完对抗生成网络后可以加深对对抗自编码器的理解。


## 12.4 变分自编码器
基本的自编码器本质上是学习输入$x$和隐藏变量$z$之间映射关系，它是一个`判别模型`(Discriminative model)，并不是`生成模型`(Generative model)。那么能不能将自编码器调整为生成模型，方便地生成样本呢？

给定隐藏变量的分布$P(z)$，如果可以学习到条件概率分布$P(x|z)$，则通过对联合概率分布$P(x,z) = P(x|z)P(z)$进行采样，生成不同的样本。`变分自编码器`(Variational AutoEncoders，简称`VAE`)就可以实现此目的，如`图12.11`所示。

<img src="images/12_11.png" style="width:500px;"/>

如果从神经网络的角度来理解的话，`VAE`和前面的自编码器一样，非常直观好理解；但是`VAE`的理论推导稍复杂，接下来我们先从神经网络的角度去阐述`VAE`，再从概率角度去推导`VAE`。

从神经网络的角度来看，`VAE`相对于自编码器模型，同样具有编码器和解码器两个子网络。解码器接受输入$x$，输出为隐变量$z$；解码器负责将隐变量$z$解码为重建的$x$。不同的是，`VAE`模型对隐变量$z$的分布有显式地约束，希望隐变量$z$符合预设的先验分布$P(z)$。因此，在损失函数的设计上，除了原有的重建误差项外，还添加了隐变量$z$分布的约束项。

**关于`变分自编码器`后续内容用到再说，本章就读到这里吧！**